<a href="https://colab.research.google.com/github/klarcika/TZ/blob/main/naloga5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow

In [4]:
#1. Izpišite koliko je vseh slik v mapi data.
import pathlib
import numpy as np
import PIL

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

pot = pathlib.Path("data")
st_slik = len(list(pot.glob('*/*.jpg')))
print("Število vseh slik: " + str(st_slik))

In [5]:
#2. Izrišite graf na katerem prikažete koliko je slik v posamezni kategoriji oz.
# koliko slik pripada posameznemu razredu
import matplotlib.pyplot as plt
from pathlib import Path

# Seznam map za različne kategorije
kategorije = ["cat", "dog", "flower", "human"]

# Seznam za shranjevanje števila slik v vsaki kategoriji
stevilo_slik = []

# Preštejte število slik v vsaki kategoriji
for kategorija in kategorije:
    slike = list(pot.glob(f"{kategorija}/*"))
    stevilo_slik.append(len(slike))

# Ustvarite stolpčni graf
plt.bar(kategorije, stevilo_slik)

# Nastavite naslov in oznake osi
plt.title("Število slik po kategorijah")
plt.xlabel("Kategorije")
plt.ylabel("Število slik")

# Prikažite graf
plt.show()


In [6]:
#3. Preverite kakšne slike so shranjene v obstoječih mapah,
# tako da izpišete po 3 primerke za vsak posamezen razred.
for j in range(kategorije.length):
  for i in range(3):
      display(PIL.Image.open(str(kategorije[j][i])))
      i=i+1
      j=j+1;

In [7]:
#4. Obstoječe podatke (iz mape data) razdelite v testni in
# validacijski dataset, glede na naključno stanje 123. Velikost učne množice naj bo 70%.
# Vsem slikam spremenite velikost na 120x120. Velikost serije (batch size) naj bo 25.
img_height = 120
img_width = 120
batch_size = 25
#razdelitev podatkov
train = tf.keras.preprocessing.image_dataset_from_directory(pot,
                                                            validation_split=0.3, #velikost validacijske množice je 30%
                                                            subset="training", #povemo da tu želimo pridobiti slike za učno množico
                                                            seed=123,
                                                            image_size=(img_height, img_width), #spremenimo velikost slik
                                                            batch_size=batch_size) #nastavimo velikost serije

In [ ]:
validation = tf.keras.preprocessing.image_dataset_from_directory(pot,
                                                                 validation_split=0.3, #velikost validacijske množice je 30%
                                                                 subset="validation", #povemo da tu želimo pridobiti slike za validacijsko množico
                                                                 seed=123,
                                                                 image_size=(img_height, img_width), #spremenimo velikost slik
                                                                 batch_size=batch_size) #nastavimo velikost serije

In [ ]:
#5. Prikažite še 15 slik s pripadajočimi razredi iz učne množice.
import matplotlib.pyplot as plt
imena_razredov = train.class_names
plt.figure(figsize=(15, 15)) #nastavimo velikost slik za prikaz
for images, labels in train.take(1):
    for i in range(15): #izpišemo 10 slik
        ax = plt.subplot(2, 5, i + 1) #izpišemo 5 slik v eni vrstici
        plt.imshow(images[i].numpy().astype("uint8")) #prikažemo sliko
        plt.title(imena_razredov[labels[i]]) #pikažemo napis nad sliko

In [ ]:
#6. Izpišite kakšni so izhodni razredi slik iz prve serije (batch-a).
for image_batch, labels_batch in train:
    print(labels_batch.shape)
    print(labels_batch) #izpis razredov za prvo serijo (batch)
    break

In [ ]:
# 7. Nad slikami izvedite augmentacijo tako da jih obrnete glede na
#  vertikalno os ter jih zavrtite in približate za vrednost 0,3.
# Augmentacijo lahko izvedete predhodna nad slikami, ali jo vključite v model.
# Pazite, da jo izvedete samo enkrat.
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("vertical", input_shape=(img_height, img_width, 3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.3),
])

plt.figure(figsize=(15, 15))
for images, _ in train.take(1):
    for i in range(3):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(1, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))


In [ ]:
#8. Standardizirajte podatke (standardizacijo lahko izvedete
#neposredno nad datasetom s slikami
#ali pa jo vključite kot samostojno raven v model).
st_razredov = 3
model = Sequential()
model.add(layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, img_width, 3))) # Data augmentation
model.add(layers.experimental.preprocessing.RandomRotation(0.1)) # Data augmentation
model.add(layers.experimental.preprocessing.RandomZoom(0.1)) # Data augmentation
model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3))) # Standardizacija podatkov
model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(st_razredov))


In [ ]:
# 9. Ustvarite Sequential model, ki bo vseboval dva bloka
#  s konvolucijskimi nevronskimi mrežami.
# Prvi blok naj ima vrednost filters nastavljeno na 32,
# drugi pa na 64. kernel_size naj bo pri obeh 3,
# prav tako naj imata ob aktivacijsko funkcijo 'relu',
# vsakemu pa naj sledi raven MaxPooling2D.
# Za zadnjim MaxPooling-om dodajte Dropout raven z vrednostjo 0,2.
# Kot zadnji dodajte še Flatten in Dense raven.
model = Sequential()

# Prvi konvolucijski blok
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D((2, 2)))

# Drugi konvolucijski blok
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.2))  # Dodamo Dropout plast
model.add(Flatten())  # Flatten plast
model.add(Dense(st_razredov))  # Polno povezana plast na koncu
